# IBM Professional Certificate on Data Science - Capstone Project

**Description:**
*Entering the coffee shop market in Jakarta, Indonesia*

This project tried to figure out the best location to open a new coffee shop around Jakarta, Indonesia based on its coffee shop density (so that it will be less competition). 

**Table of contents:**
1. Installing libraries & dependencies
2. Importing libraries
3. Data scraping
4. Getting the geo-locations
5. Mapping the neighborhood
6. Foursquare API to explore neighborhood
7. Checking out each neighborhood
8. Clustering neighborhood using K-Means
9. Map the result
10. Analyze the cluster
11. Discussion

## 1. Installing libraries & dependencies

In [1]:
!pip install geocoder

In [2]:
!pip install folium

## 2. Importing libraries

In [3]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)

import json # library to handle JSON files

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
import geocoder # to get coordinates

import requests # library to handle requests
from bs4 import BeautifulSoup # library to parse HTML and XML documents

from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

import folium # map rendering library

print("Libraries imported.")

Libraries imported.


## 3. Data scraping

In [4]:
# Request data from wikipedia
data = requests.get("https://en.wikipedia.org/wiki/Category:Districts_of_Jakarta").text # District data

In [5]:
# Parsing data
soup = BeautifulSoup(data, 'html.parser')

In [6]:
# create an empty list
neighborhoodList = []

In [7]:
# Fill the data to the list
for row in soup.find_all("div", class_="mw-category")[0].findAll("li"):
    neighborhoodList.append(row.text)

In [8]:
# Convert list to dataframe
jkt = pd.DataFrame({"Neighborhood": neighborhoodList})

jkt.head()

,Neighborhood
0,List of districts of Jakarta
1,List of Jakarta Subdistricts
2,Cakung
3,Cempaka Putih
4,Cengkareng


In [9]:
# Drop first 2 value
jkt = jkt[2:]

jkt.head()

,Neighborhood
2,Cakung
3,Cempaka Putih
4,Cengkareng
5,Cilandak
6,Cilincing


In [10]:
# Total collected neighborhood in Jakarta
print("In total, this dataset will contain:", jkt.shape[0], 'districts')

In total, this dataset will contain: 44 districts


## 4. Getting the geo-locations

In [11]:
# define a function to get location coordinates
geolocator = Nominatim(user_agent="my-application")
def get_latlng(neighborhood):
    # initialize your variable to None
    lat_lng_coords = None
    # loop until you get the coordinates
    while(lat_lng_coords is None):
        g = geocoder.arcgis('{}'.format(neighborhood))
        lat_lng_coords = g.latlng
    return lat_lng_coords

In [12]:
# call the function to get the coordinates, store in a new list using list comprehension
coords = [ get_latlng(neighborhood) for neighborhood in jkt["Neighborhood"].tolist() ]

In [13]:
# create temporary dataframe to populate the coordinates into Latitude and Longitude
df_coords = pd.DataFrame(coords, columns=['Latitude', 'Longitude'])

In [14]:
# merge the coordinates into the original dataframe
jkt['Latitude'] = df_coords['Latitude']
jkt['Longitude'] = df_coords['Longitude']

In [15]:
# check the neighborhoods and the coordinates
print(jkt.shape)
jkt

(44, 3)


,Neighborhood,Latitude,Longitude
2,Cakung,-6.13060,106.74559
3,Cempaka Putih,-6.29051,106.79491
4,Cengkareng,-6.42750,106.66556
5,Cilandak,-6.43768,106.80755
6,Cilincing,-6.32879,106.88475
7,Cipayung,-6.23501,106.92261
8,Ciracas,-6.17299,106.81571
9,Duren Sawit,-6.16777,106.78460
10,"Gambir, Jakarta",-6.34308,106.81745
11,Grogol Petamburan,-7.08638,109.21602


There are two missing geo-locations. I will troubleshoot it.

In [16]:
# Finding out the geo-locations for missing data
miss_address1 = 'Tanjung Priok, Jakarta, Indonesia'
miss_address2 = 'Tebet, South Jakarta, Indonesia'

geolocator = Nominatim(user_agent="my-application")
location1 = geolocator.geocode(miss_address1)
latitude1 = location1.latitude
longitude1 = location1.longitude
location2 = geolocator.geocode(miss_address2)
latitude2 = location2.latitude
longitude2 = location2.longitude
print('The geograpical coordinates of  Tanjung Priok {}, {}.'.format(latitude1, longitude1))
print('The geograpical coordinates of Tebet  {}, {}.'.format(latitude1, longitude1))

The geograpical coordinates of  Tanjung Priok -6.12885785, 106.87079323031739.
The geograpical coordinates of Tebet  -6.12885785, 106.87079323031739.


In [17]:
# Fill it into dataset
jkt.Latitude[jkt['Neighborhood']=='Tanjung Priok'] = latitude1
jkt.Longitude[jkt['Neighborhood']=='Tanjung Priok'] = longitude1
jkt.Latitude[jkt['Neighborhood']=='Tebet, South Jakarta'] = latitude2
jkt.Longitude[jkt['Neighborhood']=='Tebet, South Jakarta'] = longitude2

/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app
/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()
/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/opt/conda/envs/Python36/lib/python3.6/site-packages/ip

In [18]:
# Check the result
jkt[-2:]

,Neighborhood,Latitude,Longitude
44,Tanjung Priok,-6.128858,106.870793
45,"Tebet, South Jakarta",-6.226016,106.858396


**Sweet..**

In [19]:
# save the DataFrame as CSV file
jkt.to_csv("jkt.csv", index=False)

## 5. Mapping the neighborhood

In [20]:
# get the coordinates of Kuala Lumpur
address = 'Jakarta, Indonesia'

geolocator = Nominatim(user_agent="my-application")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of DKI Jakarta, Indonesia  {}, {}.'.format(latitude, longitude))

The geograpical coordinate of DKI Jakarta, Indonesia  -6.1753942, 106.827183.


In [21]:
# create map of Toronto using latitude and longitude values
map_jkt = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, neighborhood in zip(jkt['Latitude'], jkt['Longitude'], jkt['Neighborhood']):
    label = '{}'.format(neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='red',
        fill_opacity=0.7).add_to(map_jkt)  
    
map_jkt

In [22]:
# save the map as HTML file
map_jkt.save('map_jkt.html')

## 6. Foursquare API to explore neighborhood

In [45]:
# Insert credentials
CLIENT_ID = 'SECRET' # your Foursquare ID
CLIENT_SECRET = 'SECRET' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: SECRET
CLIENT_SECRET:SECRET


In [24]:
radius = 2000
LIMIT = 100

venues = []

for lat, long, neighborhood in zip(jkt['Latitude'], jkt['Longitude'], jkt['Neighborhood']):
    
    # create the API request URL
    url = "https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}".format(
        CLIENT_ID,
        CLIENT_SECRET,
        VERSION,
        lat,
        long,
        radius, 
        LIMIT)
    
    # make the GET request
    results = requests.get(url).json()["response"]['groups'][0]['items']
    
    # return only relevant information for each nearby venue
    for venue in results:
        venues.append((
            neighborhood,
            lat, 
            long, 
            venue['venue']['name'], 
            venue['venue']['location']['lat'], 
            venue['venue']['location']['lng'],  
            venue['venue']['categories'][0]['name']))

In [25]:
# convert the venues list into a new DataFrame
venues_df = pd.DataFrame(venues)

# define the column names
venues_df.columns = ['Neighborhood', 'Latitude', 'Longitude', 'VenueName', 'VenueLatitude', 'VenueLongitude', 'VenueCategory']

print(venues_df.shape)
venues_df.head()

(3074, 7)


,Neighborhood,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
0,Cakung,-6.1306,106.74559,Ikana - Ikana Seafood & Cafe,-6.124557,106.749859,Seafood Restaurant
1,Cakung,-6.1306,106.74559,Bakso Aan,-6.123921,106.743012,Asian Restaurant
2,Cakung,-6.1306,106.74559,MYSTIQUE Pool - Lounge - Dine,-6.123594,106.740567,Sports Bar
3,Cakung,-6.1306,106.74559,Uncle Tjhin Bistro,-6.123666,106.741271,Bistro
4,Cakung,-6.1306,106.74559,Fins Recipe,-6.124033,106.743319,Dessert Shop


In [26]:
# Number of venue per neighborhood
venues_df.groupby(["Neighborhood"]).count()

,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
Neighborhood,,,,,,
Cakung,65,65,65,65,65,65
Cempaka Putih,100,100,100,100,100,100
Cengkareng,1,1,1,1,1,1
Cilandak,5,5,5,5,5,5
Cilincing,32,32,32,32,32,32
Cipayung,80,80,80,80,80,80
Ciracas,100,100,100,100,100,100
Duren Sawit,100,100,100,100,100,100
"Gambir, Jakarta",50,50,50,50,50,50


In [27]:
# print out the list of categories
venues_df['VenueCategory'].unique()[:20]

array(['Seafood Restaurant', 'Asian Restaurant', 'Sports Bar', 'Bistro',
       'Dessert Shop', 'Noodle House', 'Steakhouse', 'Bar', 'Golf Course',
       'Housing Development', 'Pool', 'Coffee Shop',
       'Japanese Restaurant', 'Indonesian Restaurant', 'Farmers Market',
       'Malay Restaurant', 'Chinese Restaurant', 'Water Park',
       'Food Court', 'Pool Hall'], dtype=object)

In [28]:
print('There are {} uniques categories.'.format(len(venues_df['VenueCategory'].unique())))

There are 248 uniques categories.


## 7. Check out each neighborhood

In [29]:
# One Hot Encoding
jkt_onehot = pd.get_dummies(venues_df[['VenueCategory']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
jkt_onehot['Neighborhoods'] = venues_df['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [jkt_onehot.columns[-1]] + list(jkt_onehot.columns[:-1])
jkt_onehot = jkt_onehot[fixed_columns]

print(jkt_onehot.shape)
jkt_onehot.head()

(3074, 249)


,Neighborhoods,Accessories Store,Acehnese Restaurant,African Restaurant,Airport,American Restaurant,Aquarium,Arcade,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Dealership,Automotive Shop,BBQ Joint,Badminton Court,Bakery,Balinese Restaurant,Bank,Bar,Baseball Stadium,Basketball Court,Beach,Bed & Breakfast,Beer Bar,Beer Garden,Betawinese Restaurant,Big Box Store,Bistro,Board Shop,Boarding House,Bookstore,Boutique,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Bubble Tea Shop,Buffet,Building,Burger Joint,Bus Station,Cable Car,Cafeteria,Café,Camera Store,Cantonese Restaurant,Car Wash,Cheese Shop,Chinese Restaurant,Church,Clothing Store,Coffee Shop,College Academic Building,College Cafeteria,College Residence Hall,Comedy Club,Comfort Food Restaurant,Concert Hall,Convenience Store,Cosmetics Shop,Creperie,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Dog Run,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Elementary School,Event Space,Fabric Shop,Factory,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Stand,Food Truck,Football Stadium,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Garden,Gas Station,Gastropub,General Entertainment,German Restaurant,Gift Shop,Golf Course,Gourmet Shop,Government Building,Grocery Store,Gun Range,Gym,Gym / Fitness Center,Gym Pool,Halal Restaurant,Harbor / Marina,Hardware Store,Health & Beauty Service,High School,Historic Site,History Museum,Hobby Shop,Hookah Bar,Hospital,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Housing Development,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Meatball Place,Indonesian Restaurant,Indoor Play Area,Italian Restaurant,Japanese Restaurant,Javanese Restaurant,Jewelry Store,Juice Bar,Karaoke Bar,Kebab Restaurant,Korean Restaurant,Lake,Lounge,Malay Restaurant,Manadonese Restaurant,Market,Massage Studio,Medical Center,Mediterranean Restaurant,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Mosque,Motorcycle Shop,Movie Theater,Multiplex,Museum,Music School,Music Store,Music Venue,Nightclub,Noodle House,Office,Other Great Outdoors,Outdoors & Recreation,Padangnese Restaurant,Paper / Office Supplies Store,Park,Performing Arts Venue,Pet Store,Pharmacy,Photography Studio,Piano Bar,Pizza Place,Planetarium,Playground,Plaza,Pool,Pool Hall,Pub,Public Art,Racetrack,Ramen Restaurant,Recreation Center,Residential Building (Apartment / Condo),Resort,Rest Area,Restaurant,River,Salad Place,Salon / Barbershop,Sandwich Place,Satay Restaurant,Science Museum,Sculpture Garden,Seafood Restaurant,Shabu-Shabu Restaurant,Shopping Mall,Shopping Plaza,Skating Rink,Snack Place,Soccer Field,Soccer Stadium,Soup Place,Southern / Soul Food Restaurant,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Steakhouse,Street Food Gathering,Student Center,Sundanese Restaurant,Supermarket,Surf Spot,Sushi Restaurant,Tapas Restaurant,Tea Room,Tex-Mex Restaurant,Thai Restaurant,Theater,Theme Park,Theme Park Ride / Attraction,Theme Restaurant,Toy / Game Store,Track,Track Stadium,Trail,Train,Train Station,Travel Agency,Udon Restaurant,University,Vacation Rental,Vape Store,Vegetarian / Vegan Restaurant,Veterinarian,Video Game Store,Vietnamese Restaurant,Water Park,Wine Bar,Winery,Wings Joint,Women's Store,Zoo
0,Cakung,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0

In [30]:
jkt_grouped = jkt_onehot.groupby(["Neighborhoods"]).mean().reset_index()

print(jkt_grouped.shape)
jkt_grouped

(41, 249)


,Neighborhoods,Accessories Store,Acehnese Restaurant,African Restaurant,Airport,American Restaurant,Aquarium,Arcade,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Dealership,Automotive Shop,BBQ Joint,Badminton Court,Bakery,Balinese Restaurant,Bank,Bar,Baseball Stadium,Basketball Court,Beach,Bed & Breakfast,Beer Bar,Beer Garden,Betawinese Restaurant,Big Box Store,Bistro,Board Shop,Boarding House,Bookstore,Boutique,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Bubble Tea Shop,Buffet,Building,Burger Joint,Bus Station,Cable Car,Cafeteria,Café,Camera Store,Cantonese Restaurant,Car Wash,Cheese Shop,Chinese Restaurant,Church,Clothing Store,Coffee Shop,College Academic Building,College Cafeteria,College Residence Hall,Comedy Club,Comfort Food Restaurant,Concert Hall,Convenience Store,Cosmetics Shop,Creperie,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Dog Run,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Elementary School,Event Space,Fabric Shop,Factory,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Stand,Food Truck,Football Stadium,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Garden,Gas Station,Gastropub,General Entertainment,German Restaurant,Gift Shop,Golf Course,Gourmet Shop,Government Building,Grocery Store,Gun Range,Gym,Gym / Fitness Center,Gym Pool,Halal Restaurant,Harbor / Marina,Hardware Store,Health & Beauty Service,High School,Historic Site,History Museum,Hobby Shop,Hookah Bar,Hospital,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Housing Development,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Meatball Place,Indonesian Restaurant,Indoor Play Area,Italian Restaurant,Japanese Restaurant,Javanese Restaurant,Jewelry Store,Juice Bar,Karaoke Bar,Kebab Restaurant,Korean Restaurant,Lake,Lounge,Malay Restaurant,Manadonese Restaurant,Market,Massage Studio,Medical Center,Mediterranean Restaurant,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Mosque,Motorcycle Shop,Movie Theater,Multiplex,Museum,Music School,Music Store,Music Venue,Nightclub,Noodle House,Office,Other Great Outdoors,Outdoors & Recreation,Padangnese Restaurant,Paper / Office Supplies Store,Park,Performing Arts Venue,Pet Store,Pharmacy,Photography Studio,Piano Bar,Pizza Place,Planetarium,Playground,Plaza,Pool,Pool Hall,Pub,Public Art,Racetrack,Ramen Restaurant,Recreation Center,Residential Building (Apartment / Condo),Resort,Rest Area,Restaurant,River,Salad Place,Salon / Barbershop,Sandwich Place,Satay Restaurant,Science Museum,Sculpture Garden,Seafood Restaurant,Shabu-Shabu Restaurant,Shopping Mall,Shopping Plaza,Skating Rink,Snack Place,Soccer Field,Soccer Stadium,Soup Place,Southern / Soul Food Restaurant,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Steakhouse,Street Food Gathering,Student Center,Sundanese Restaurant,Supermarket,Surf Spot,Sushi Restaurant,Tapas Restaurant,Tea Room,Tex-Mex Restaurant,Thai Restaurant,Theater,Theme Park,Theme Park Ride / Attraction,Theme Restaurant,Toy / Game Store,Track,Track Stadium,Trail,Train,Train Station,Travel Agency,Udon Restaurant,University,Vacation Rental,Vape Store,Vegetarian / Vegan Restaurant,Veterinarian,Video Game Store,Vietnamese Restaurant,Water Park,Wine Bar,Winery,Wings Joint,Women's Store,Zoo
0,Cakung,0.00,0.00,0.00,0.00,0.00,0.00,0.000000,0.00,0.00,0.00,0.123077,0.00,0.000000,0.0000,0.000000,0.00000,0.000000,0.00,0.015385,0.030769,0.000000,0.00,0.00,0.00,0.015385,0.00,0.00,0.0000,0.030769,0.00,0.00,0.015385,0.00,0.000000,0.00,0.000000,0.015385,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.046154,0.00,0.00,0.00,0.000000,0.030769,0.00,0.015385,0.046154,0.00,0.00,0.00,0.00,0.000000,0.00,0.03

In [31]:
print('Coffeeshop is distributed around:',len(jkt_grouped[jkt_grouped["Coffee Shop"] > 0]), 'of 44 districts')

Coffeeshop is distributed around: 31 of 44 districts


In [32]:
jkt_coffee = jkt_grouped[["Neighborhoods","Coffee Shop"]]

In [33]:
jkt_coffee.head()

,Neighborhoods,Coffee Shop
0,Cakung,0.046154
1,Cempaka Putih,0.180000
2,Cengkareng,0.000000
3,Cilandak,0.000000
4,Cilincing,0.093750


In [34]:
jkt_coffee

,Neighborhoods,Coffee Shop
0,Cakung,0.046154
1,Cempaka Putih,0.180000
2,Cengkareng,0.000000
3,Cilandak,0.000000
4,Cilincing,0.093750
5,Cipayung,0.025000
6,Ciracas,0.090000
7,Duren Sawit,0.110000
8,"Gambir, Jakarta",0.000000
9,Grogol Petamburan,0.000000


## 8. Clustering neighborhood using K-Means

In [35]:
kclusters = 3

jkt_clustering = jkt_coffee.drop(["Neighborhoods"], 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(jkt_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([2, 1, 0, 0, 2, 0, 2, 2, 0, 0], dtype=int32)

In [36]:
# create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.
jkt_merged = jkt_coffee.copy()

# add clustering labels
jkt_merged["Cluster Labels"] = kmeans.labels_

In [37]:
jkt_merged.rename(columns={"Neighborhoods": "Neighborhood"}, inplace=True)
jkt_merged.head()

,Neighborhood,Coffee Shop,Cluster Labels
0,Cakung,0.046154,2
1,Cempaka Putih,0.180000,1
2,Cengkareng,0.000000,0
3,Cilandak,0.000000,0
4,Cilincing,0.093750,2


In [38]:
jkt_merged = jkt_merged.join(jkt.set_index("Neighborhood"), on="Neighborhood")

print(jkt_merged.shape)
jkt_merged.head() # check the last columns!

(41, 5)


,Neighborhood,Coffee Shop,Cluster Labels,Latitude,Longitude
0,Cakung,0.046154,2,-6.13060,106.74559
1,Cempaka Putih,0.180000,1,-6.29051,106.79491
2,Cengkareng,0.000000,0,-6.42750,106.66556
3,Cilandak,0.000000,0,-6.43768,106.80755
4,Cilincing,0.093750,2,-6.32879,106.88475


In [39]:
# sort the results by Cluster Labels
print(jkt_merged.shape)
jkt_merged.sort_values(["Cluster Labels"], inplace=True)
jkt_merged

(41, 5)


,Neighborhood,Coffee Shop,Cluster Labels,Latitude,Longitude
20,Kemayoran,0.000000,0,-6.127160,106.899190
32,Pejompongan,0.016949,0,-6.236400,106.754470
29,"Pancoran, South Jakarta",0.000000,0,-6.337690,106.857550
26,Matraman,0.000000,0,-7.421940,109.366940
22,"Koja, Jakarta",0.000000,0,-6.285050,106.902420
39,Tanjung Priok,0.030000,0,-6.128858,106.870793
19,Kelapa Gading,0.000000,0,-6.218230,106.737490
12,Jelambar,0.000000,0,-7.271110,109.008890
9,Grogol Petamburan,0.000000,0,-7.086380,109.216020
40,"Tebet, South Jakarta",0.040000,0,-6.226016,106.858396


## 9. Map the result

In [40]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(jkt_merged['Latitude'], jkt_merged['Longitude'], jkt_merged['Neighborhood'], jkt_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' - Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=['red','blue','darkgreen'][cluster-1],
        fill=True,
        fill_color=['red','blue','darkgreen'][cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [41]:
# save the map as HTML file
map_clusters.save('map_clusters.html')

## Analyze the cluster

**Cluster 0**

In [42]:
jkt_merged.loc[jkt_merged['Cluster Labels'] == 0]

,Neighborhood,Coffee Shop,Cluster Labels,Latitude,Longitude
20,Kemayoran,0.000000,0,-6.127160,106.899190
32,Pejompongan,0.016949,0,-6.236400,106.754470
29,"Pancoran, South Jakarta",0.000000,0,-6.337690,106.857550
26,Matraman,0.000000,0,-7.421940,109.366940
22,"Koja, Jakarta",0.000000,0,-6.285050,106.902420
39,Tanjung Priok,0.030000,0,-6.128858,106.870793
19,Kelapa Gading,0.000000,0,-6.218230,106.737490
12,Jelambar,0.000000,0,-7.271110,109.008890
9,Grogol Petamburan,0.000000,0,-7.086380,109.216020
40,"Tebet, South Jakarta",0.040000,0,-6.226016,106.858396


**Cluster 1**

In [43]:
jkt_merged.loc[jkt_merged['Cluster Labels'] == 1]

,Neighborhood,Coffee Shop,Cluster Labels,Latitude,Longitude
23,"Kramat Jati, East Jakarta",0.14,1,-6.24525,106.83148
28,Pademangan,0.13,1,-6.25548,106.84391
35,Sawah Besar,0.17,1,-6.22124,106.82528
1,Cempaka Putih,0.18,1,-6.29051,106.79491


**Cluster 2**

In [44]:
jkt_merged.loc[jkt_merged['Cluster Labels'] == 2]

,Neighborhood,Coffee Shop,Cluster Labels,Latitude,Longitude
27,Menteng,0.110000,2,-6.19153,106.79556
10,Jagakarsa,0.080000,2,-6.16040,106.78470
33,"Pesanggrahan, South Jakarta",0.050000,2,-6.15741,106.83140
34,Pulo Gadung,0.090000,2,-6.19675,106.84848
36,Senen,0.050000,2,-6.15327,106.82357
37,"Taman Sari, Jakarta",0.050000,2,-6.12842,106.85952
38,Tanah Abang,0.090000,2,-6.22939,106.84971
30,Pasar Minggu,0.080000,2,-6.20139,106.80167
31,Pasar Rebo,0.080000,2,-6.11662,106.77803
25,Mampang Prapatan,0.100000,2,-6.20311,106.83798


## Discussion

This research attempts to investigate the distribution of coffeeshop around DKI Jakarta; also, it figures out the best location to open a new coffeeshop considering the its density in Jakarta. Simply, the best location to open new coffeeshop is determined by finding out the locations that is relatively containing less coffee shop. Therefore, by applying a clustering method, I try to cluster jakarta districts by three region. **Cluster 0** is a cluster that contain some cofee shops. Secondly, **Cluster 1** is a cluster that contain less coffee shops, where relatively located at uptown. Finally, **Cluster 2** is a cluster that contain the most dense coffee shop, where relatively located at downtown Jakarta.

Consequently, according to the data, the best location to open new coffeeshop is around the uptown Jakarta, where it relatively populous due to it's located at residential. Moreover, the concept of the coffee shop may be fit with the visitors' profile and its main market will be good if it focused on weekend.